In [23]:
import numpy as np
import json
import os, sys
import pandas as pd
import glob
import cv2

In [197]:
def createCOCODataset(annotation_file, project_name='my-projecte-name'):
    '''create list[dict] as COCO dataset from makesense.ai keypoint .csv output'''
    
    image_labels = pd.read_csv(annotation_file, header=None).to_numpy()
    labels = np.unique(image_labels[:, 0])
    category_idx = np.unique(image_labels[:, 6])
    category_labels = np.unique(image_labels[:, 7])

    dataset = {
        "info": {
            "description": str(project_name)
        },
        "categories": [{
            "id": category_idx[i],
            "name": category_labels[i]
        } for i in np.arange(len(category_idx))],
        "images": [{
            "id":
            i,
            "width":
            int(image_labels[image_labels[:, 3] == image][:, 4][0]),
            "height":
            int(image_labels[image_labels[:, 3] == image][:, 5][0]),
            "file_name":
            str(image.replace('jpg','png'))
        } for i, image in enumerate(np.unique(image_labels[:, 3]))],
        "annotations": []
#         "keypoint_names": list(np.unique(image_labels[:,0]))
    }

    for i, image in enumerate(np.unique(image_labels[:, 3])):
        subset = image_labels[image_labels[:, 3] == image]
        width = subset[:, 4][0]
        height = subset[:, 5][0]

        # category probably equivalent to class
        for category_idx in np.unique(subset[:, 6]):
            subsubset = subset[subset[:, 6] == category_idx]
            pts = np.array(subsubset[:, 1:3]).astype(int)
            x, y, w, h = cv2.boundingRect(pts)
            image_id = int(np.where(np.unique(image_labels[:, 3])==subsubset[:,3][0])[0])
            keypoints = np.concatenate([
                np.c_[subsubset[subsubset[:, 0] == i][:, 1:3],
                      1] if np.isin(i, subsubset[:, 0]) else [[-1, -1, 0]]
                for i in labels
            ]).reshape(-1).astype(float)
            
            entry = {
                'bbox': [float(x), float(y),
                         float(w), float(h)],
                'bbox_mode': 1,
                'id': len(dataset['annotations']),
                'image_id': image_id,
                'category_id': int(category_idx),
                'segmentation': [],
                'keypoints': list(keypoints)
            }

            dataset['annotations'].append(entry)
    return dataset

In [ ]:
raw_annotations = '/home/fritz/Downloads/labels_puffer.csv'
image_labels = pd.read_csv(raw_annotations, header=None)
## create dummy variable for class index (NOTE: This should be changed if you have multiple classes)
image_labels['6'] = np.repeat(1,len(images_labels))
## create dummy variable for class name (NOTE: This should be changed if you have multiple classes)
image_labels['7'] = np.repeat('fish',len(image_labels))
## Save to new output 
# manipulated_annoations = '/home/fritz/Downloads/labels_puffer_test.csv'
# image_labels.to_csv(manipulated_annotations, header=None, index=None)
image_labels = image_labels.to_numpy()
image_labels

In [198]:
## create COCO dataset from makesense.ai keypoint annotations
annotation_file = '/home/fritz/Downloads/labels_puffer_test.csv'
dataset = createCOCODataset(annotation_file)
dataset

{'info': {'description': 'my-projecte-name'},
 'categories': [{'id': 1, 'name': 'fish'}],
 'images': [{'id': 0,
   'width': 1920,
   'height': 1080,
   'file_name': 'output_001.png'},
  {'id': 1, 'width': 1920, 'height': 1080, 'file_name': 'output_002.png'},
  {'id': 2, 'width': 1920, 'height': 1080, 'file_name': 'output_003.png'},
  {'id': 3, 'width': 1920, 'height': 1080, 'file_name': 'output_004.png'},
  {'id': 4, 'width': 1920, 'height': 1080, 'file_name': 'output_005.png'},
  {'id': 5, 'width': 1920, 'height': 1080, 'file_name': 'output_006.png'},
  {'id': 6, 'width': 1920, 'height': 1080, 'file_name': 'output_007.png'},
  {'id': 7, 'width': 1920, 'height': 1080, 'file_name': 'output_008.png'},
  {'id': 8, 'width': 1920, 'height': 1080, 'file_name': 'output_009.png'},
  {'id': 9, 'width': 1920, 'height': 1080, 'file_name': 'output_010.png'}],
 'annotations': [{'bbox': [410.0, 276.0, 1407.0, 325.0],
   'bbox_mode': 1,
   'id': 0,
   'image_id': 0,
   'category_id': 1,
   'segmentat

In [199]:
## save to .json file
## this can be used with Detectron2 on Google Colab

json_object = json.dumps(dataset, indent = 4) 
output_name = '/home/fritz/Downloads/pufferDetectron2.json'
with open(output_name, 'w') as f:
    json.dump(dataset, f, indent=4)